In [1]:
import os
import json
import time
import random
import csv
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Any
from dotenv import load_dotenv

# --- IMPORTAÇÃO DA SDK (google-genai) ---
from google import genai
from google.genai import types

# ==============================================================================
# CONFIGURAÇÕES GERAIS
# ==============================================================================

# Carrega variáveis do arquivo .env
load_dotenv()

# 1. Configuração da API Key
API_KEY = os.getenv("GEMINI_API_KEY")

# 2. Configuração do Modelo
MODEL_NAME = "gemini-2.5-flash"

# 3. Configuração de Geração Padrão
GENERATION_CONFIG = types.GenerateContentConfig(
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    #max_output_tokens=1000,
    response_mime_type="text/plain",
    safety_settings=[
        types.SafetySetting(
            category="HARM_CATEGORY_HATE_SPEECH",
            threshold="BLOCK_NONE"
        ),
        types.SafetySetting(
            category="HARM_CATEGORY_DANGEROUS_CONTENT",
            threshold="BLOCK_NONE"
        ),
        types.SafetySetting(
            category="HARM_CATEGORY_HARASSMENT",
            threshold="BLOCK_NONE"
        ),
        types.SafetySetting(
            category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
            threshold="BLOCK_NONE"
        ),
    ]
)

# 4. Configuração de Amostragem e Logs
QTDE_AMOSTRA_TESTE = 1  # Mantido conforme seu notebook

# Caminhos (Ajustados conforme seu notebook)
INPUT_DIR = Path('data/json/test_ready')
FEW_SHOT_DIR = Path('data/json/fewshot_samples')
OUTPUT_FILE = Path('results/cenipa_gemini_results.json')
LOG_FILE = Path('logs/log_execucao_arquivos.csv')

# ==============================================================================
# TAXONOMIA CENIPA (Do seu arquivo base)
# ==============================================================================
TAXONOMIA_CENIPA = {
    "Fatores Humanos": [
        "Álcool", "Ansiedade", "Atenção", "Atitude", "Capacitação e treinamento",  "Características da tarefa", "Clima organizacional",
        "Comunicação", "Condições físicas do trabalho", "Cultura do grupo de trabalho", "Cultura organizacional", "Desorientação", "Dieta inadequada",
        "Dinâmica da equipe", "Disbarismo", "Dor", "Enfermidade", "Enjoo aéreo", "Equipamento - características ergonômicas", "Estado emocional", "Fadiga",
        "Gravidez", "Hiperventilação", "Hipóxia", "Ilusões visuais", "Inconsciência", "Influências externas", "Insônia", "Intoxicação alimentar",
        "Intoxicação por CO", "Memória", "Motivação", "Obesidade", "Organização do trabalho", "Percepção", "Processo decisório", "Processos Organizacionais",
        "Próteses", "Relações interpessoais", "Ressaca", "Sistemas de apoio", "Sobrecarga de tarefas", "Uso de Medicamento", "Uso ilícito de drogas", "Vertigem",
        "Vestimenta inadequada"
    ],
    "Fatores Operacionais": [
        "Aplicação do comando", "Condições meteorológicas adversas", "Conhecimento de normas (ATS)", "Console (ATS)", "Coordenação de cabine",
        "Coordenação de tráfego (ATS)", "Desvio de navegação", "Emprego de meios (ATS)", "Equipamento de apoio (ATS)", "Fraseologia da tripulação",
        "Fraseologia do Órgão ATS", "Habilidade de controle (ATS)", "Indisciplina de voo", "Infraestrutura aeroportuária", "Instrução",
        "Julgamento de pilotagem", "Limite de autorização", "Manutenção da aeronave", "Outro", "Pessoal de apoio", "Planejamento de tráfego (ATS)",
        "Planejamento do voo", "Planejamento gerencial", "Pouca experiência do piloto", "Presença de ave", "Presença de fauna (não ave)", "Publicações (ATS)",
        "RADAR (ATS)", "Serviço fixo (ATS)", "Serviço móvel (ATS)", "Substituição na posição (ATS)", "Supervisão (ATS)", "Supervisão gerencial",
        "Tratamento (ATS)", "Visualização (ATS)"
    ],
    "Fatores Materiais": [
        "Fabricação", "Manuseio do material", "Projeto"
    ]
}

TODOS_FATORES = []
for cat in TAXONOMIA_CENIPA.values():
    TODOS_FATORES.extend(cat)

# ==============================================================================
# FUNÇÕES DE UTILIDADE
# ==============================================================================

def get_gemini_client():
    if not API_KEY:
        print("⚠️ ERRO: API Key não encontrada.")
        return None
    return genai.Client(api_key=API_KEY)

def carregar_relatorios(diretorio: Path) -> List[Dict]:
    relatorios = []
    if not diretorio.exists():
        print(f"⚠️ Diretório não encontrado: {diretorio}")
        return []

    arquivos = list(diretorio.glob("*.json"))
    if not arquivos:
        print(f"⚠️ Nenhum arquivo JSON encontrado em: {diretorio}")
        return []

    print(f"Carregando {len(arquivos)} arquivos de treino de: {diretorio}...")
    for arquivo in arquivos:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                data = json.load(f)
                data['filename'] = arquivo.name
                if 'conteudo' in data:
                    relatorios.append(data)
        except Exception as e:
            print(f"  Erro ao ler {arquivo.name}: {e}")
    return relatorios

def salvar_log_csv(relatorios_selecionados: List[Dict]):
    # Garante que a pasta logs existe
    LOG_FILE.parent.mkdir(parents=True, exist_ok=True)
    print(f"📝 Salvando log dos arquivos selecionados em: {LOG_FILE}")

    with open(LOG_FILE, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Data_Hora', 'Arquivo_Selecionado'])
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        for relatorio in relatorios_selecionados:
            writer.writerow([timestamp, relatorio['filename']])

def preparar_texto_relatorio(conteudo_json: Dict) -> str:
    historico = conteudo_json.get('historico_voo', '')
    analise = conteudo_json.get('analise', '')
    return f"""
    --- INÍCIO DO RELATÓRIO ---
    HISTÓRICO DO VOO:
    {historico}

    ANÁLISE:
    {analise}
    --- FIM DO RELATÓRIO ---
    """

def chamar_gemini(client, prompt: str, retries=3, custom_config=None) -> str:
    """
    Wrapper com DEBUG DE SEGURANÇA/BLOQUEIO.
    Se a resposta vier vazia, investiga se foi Safety, Recitation ou Outro.
    """
    config_to_use = custom_config if custom_config else GENERATION_CONFIG

    for i in range(retries):
        try:
            response = client.models.generate_content(
                model=MODEL_NAME,
                contents=prompt,
                config=config_to_use
            )

            # Se tiver texto, sucesso!
            if response.text:
                return response.text

            # --- DEBUG: Investigando Bloqueio ---
            print(f"  ⚠️ AVISO: Resposta vazia (Tentativa {i+1}). Investigando motivo...")

            if response.candidates:
                candidate = response.candidates[0]
                reason = candidate.finish_reason
                print(f"     -> Motivo do Bloqueio (Finish Reason): {reason}")

                # Se for segurança, detalha qual filtro
                if reason == "SAFETY":
                    for r in candidate.safety_ratings:
                        # Mostra apenas os que não são NEGLIGIBLE para não poluir
                        if r.probability != "NEGLIGIBLE":
                            print(f"     -> Filtro Ativado: {r.category} | Probabilidade: {r.probability}")
                elif reason == "RECITATION":
                    print("     -> Bloqueio por Recitação (Copyright/Plágio de texto existente).")
                elif reason == "OTHER":
                    print("     -> Bloqueio 'OTHER'. Pode ser idioma ou filtro interno não especificado.")
            else:
                print("     -> Nenhum candidato retornado pela API (Erro desconhecido).")

            return "BLOQUEADO_PELO_FILTRO"

        except Exception as e:
            wait_time = 10 * (i + 1)
            msg_erro = str(e)
            if "429" in msg_erro or "RESOURCE_EXHAUSTED" in msg_erro:
                print(f"  ⚠️ COTA EXCEDIDA (429). Aguardando {wait_time}s...")
            else:
                print(f"  ⚠️ Erro na API (tentativa {i+1}/{retries}): {e}. Aguardando {wait_time}s...")
            time.sleep(wait_time)

    return "ERRO_API"

# ==============================================================================
# ESTRATÉGIAS DE PROMPT
# ==============================================================================

# 1. Zero Shot
def estrategia_zero_shot(client, texto_relatorio: str) -> str:
    prompt = f"""
    Você é um especialista em segurança de voo (CENIPA).
    Leia o relatório abaixo e liste APENAS os Fatores Contribuintes.
    SEJA CONCISO. Não escreva introduções ou justificativas. Apenas os nomes dos fatores separados por vírgula.

    {texto_relatorio}

    Fatores Contribuintes:
    """
    return chamar_gemini(client, prompt)

# 2. Few Shot
def estrategia_few_shot(client, texto_relatorio: str, exemplos: List[Dict]) -> str:
    prompt_exemplos = ""
    for ex in exemplos:
        txt_ex = preparar_texto_relatorio(ex['conteudo'])
        fatores_ex = ", ".join(ex['conteudo'].get('fatores_contribuintes', []))
        prompt_exemplos += f"""
        Exemplo de Relatório:
        {txt_ex}
        Fatores Contribuintes Identificados:
        {fatores_ex}
        -----------------------------------
        """
    prompt = f"""
    Você é um especialista CENIPA. Extraia os fatores contribuintes. Siga o padrão dos exemplos (apenas a lista).

    {prompt_exemplos}

    Novo Relatório:
    {texto_relatorio}

    Resposta (Apenas Fatores):
    """
    return chamar_gemini(client, prompt)

# 3. Auto-CoT
def estrategia_auto_cot(client, texto_relatorio: str) -> str:
    cot_config = types.GenerateContentConfig(
        temperature=0.2,
        top_p=0.8,
        top_k=40,
        #max_output_tokens=1000,
        safety_settings=GENERATION_CONFIG.safety_settings
    )
    prompt = f"""
    Você é um investigador de acidentes aéreos.
    Leia o relatório abaixo.

    {texto_relatorio}

    Tarefa: Identificar os fatores contribuintes.
    Pense passo a passo nas falhas humanas, materiais e operacionais descritas no texto.
    Raciocine sobre a causalidade.
    No final, liste os fatores contribuintes finais.
    """
    return chamar_gemini(client, prompt, custom_config=cot_config)

# 4. Zero Shot+
def estrategia_zero_shot_plus(client, texto_relatorio: str) -> str:
    lista_fatores_str = ", ".join(TODOS_FATORES)
    prompt = f"""
    Analise o relatório. Selecione quais fatores da lista oficial do CENIPA contribuíram.
    Responda APENAS com os itens da lista selecionados, separados por vírgula.

    Lista Oficial:
    [{lista_fatores_str}]

    {texto_relatorio}

    Fatores Contribuintes (apenas a lista):
    """
    return chamar_gemini(client, prompt)

# 5. CFG-CoT
def estrategia_cfg_cot(client, texto_relatorio: str) -> Dict[str, str]:
    cot_config = types.GenerateContentConfig(
        temperature=0.2,
        top_p=0.8,
        top_k=40,
        #max_output_tokens=1000,
        safety_settings=GENERATION_CONFIG.safety_settings
    )
    respostas = {}

    prompt_humanos = f"""
    Analise focando EXCLUSIVAMENTE em Fatores Humanos.
    Rótulos possíveis: {", ".join(TAXONOMIA_CENIPA['Fatores Humanos'])}.

    {texto_relatorio}

    Quais fatores humanos da lista acima estão presentes? Explique brevemente e liste.
    """
    respostas['humanos'] = chamar_gemini(client, prompt_humanos, custom_config=cot_config)

    prompt_operacionais = f"""
    Analise focando EXCLUSIVAMENTE em Fatores Operacionais.
    Rótulos possíveis: {", ".join(TAXONOMIA_CENIPA['Fatores Operacionais'])}.

    {texto_relatorio}

    Quais fatores operacionais da lista acima estão presentes? Explique brevemente e liste.
    """
    time.sleep(2)
    respostas['operacionais'] = chamar_gemini(client, prompt_operacionais, custom_config=cot_config)

    prompt_materiais = f"""
    Analise focando EXCLUSIVAMENTE em Fatores Materiais.
    Rótulos possíveis: {", ".join(TAXONOMIA_CENIPA['Fatores Materiais'])}.

    {texto_relatorio}

    Quais fatores materiais da lista acima estão presentes? Explique brevemente e liste.
    """
    time.sleep(2)
    respostas['materiais'] = chamar_gemini(client, prompt_materiais, custom_config=cot_config)

    respostas['consolidado'] = f"HUMANOS: {respostas['humanos']}\nOPERACIONAIS: {respostas['operacionais']}\nMATERIAIS: {respostas['materiais']}"
    return respostas['consolidado']

# ==============================================================================
# LOOP PRINCIPAL
# ==============================================================================

def main():
    print("--- Iniciando Benchmark CENIPA vs GEMINI (V3 - Safety Debug) ---")
    print(f"Configuração: Processar {QTDE_AMOSTRA_TESTE} arquivos aleatórios.")

    client = get_gemini_client()
    if not client:
        return

    # 1. Listar e Sortear arquivos
    print(f"\nListando arquivos no diretório: {INPUT_DIR}")
    if not INPUT_DIR.exists():
        print("❌ ERRO: Diretório de teste não encontrado.")
        return

    todos_paths = list(INPUT_DIR.glob("*.json"))
    total_disponivel = len(todos_paths)
    if total_disponivel == 0:
        print("❌ ERRO: Nenhum arquivo JSON encontrado.")
        return

    tamanho_amostra = min(QTDE_AMOSTRA_TESTE, total_disponivel)
    paths_sorteados = random.sample(todos_paths, tamanho_amostra)

    conjunto_teste = []
    for path in paths_sorteados:
        try:
            with open(path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                data['filename'] = path.name
                if 'conteudo' in data:
                    conjunto_teste.append(data)
        except Exception as e:
            print(f"  Erro ao ler {path.name}: {e}")

    if conjunto_teste:
        salvar_log_csv(conjunto_teste)
    else:
        print("Nenhum arquivo válido carregado.")
        return

    # 2. Carregar Dados de Treino
    exemplos_few_shot = carregar_relatorios(FEW_SHOT_DIR)
    if exemplos_few_shot:
        print(f"✅ {len(exemplos_few_shot)} exemplos de Few-Shot carregados.")

    print(f"\n✅ Iniciando processamento de {len(conjunto_teste)} casos de teste...")
    resultados_finais = []

    # Garante diretório de saída
    OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)

    for idx, caso in enumerate(conjunto_teste):
        nome_arq = caso.get('filename', f'caso_{idx}')
        print(f"\nProcessando caso {idx+1}/{len(conjunto_teste)}: {nome_arq}")

        texto = preparar_texto_relatorio(caso['conteudo'])
        ground_truth = caso['conteudo'].get('fatores_contribuintes', [])

        res_caso = {
            "arquivo": nome_arq,
            "ground_truth": ground_truth,
            "respostas": {}
        }

        if idx > 0:
            print("  (Aguardando 15s para aliviar cota entre arquivos...)")
            time.sleep(15)

        print("  > Executando Zero Shot...")
        res_caso["respostas"]["zero_shot"] = estrategia_zero_shot(client, texto)

        print("  > Executando Few Shot...")
        if exemplos_few_shot:
            res_caso["respostas"]["few_shot"] = estrategia_few_shot(client, texto, exemplos_few_shot)
        else:
            res_caso["respostas"]["few_shot"] = "SKIPPED_NO_EXAMPLES"

        print("  > Executando Auto-CoT...")
        res_caso["respostas"]["auto_cot"] = estrategia_auto_cot(client, texto)

        print("  > Executando Zero Shot+...")
        res_caso["respostas"]["zero_shot_plus"] = estrategia_zero_shot_plus(client, texto)

        print("  > Executando CFG-CoT (3 steps)...")
        res_caso["respostas"]["cfg_cot"] = estrategia_cfg_cot(client, texto)

        resultados_finais.append(res_caso)

    print(f"\nSalvando resultados em {OUTPUT_FILE}...")
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(resultados_finais, f, ensure_ascii=False, indent=4)

    print("Concluído com sucesso!")

if __name__ == "__main__":
    main()

--- Iniciando Benchmark CENIPA vs GEMINI (V3 - Safety Debug) ---
Configuração: Processar 1 arquivos aleatórios.

Listando arquivos no diretório: data\json\test_ready
📝 Salvando log dos arquivos selecionados em: logs\log_execucao_arquivos.csv
Carregando 3 arquivos de treino de: data\json\fewshot_samples...
✅ 3 exemplos de Few-Shot carregados.

✅ Iniciando processamento de 1 casos de teste...

Processando caso 1/1: pr_sfs_23_01_13.json
  > Executando Zero Shot...
  > Executando Few Shot...
  > Executando Auto-CoT...
  > Executando Zero Shot+...
  > Executando CFG-CoT (3 steps)...

Salvando resultados em results\cenipa_gemini_results.json...
Concluído com sucesso!
